In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv('/kaggle/input/bookname-dataset/books.csv', on_bad_lines='skip')

In [3]:
df

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [4]:
df.describe()


,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [5]:
X = df.copy()


In [6]:
X.loc[ (X['average_rating'] >= 0) & (X['average_rating'] <= 1), 'rating_between'] = "between 0 and 1"
X.loc[ (X['average_rating'] > 1) & (X['average_rating'] <= 2), 'rating_between'] = "between 1 and 2"
X.loc[ (X['average_rating'] > 2) & (X['average_rating'] <= 3), 'rating_between'] = "between 2 and 3"
X.loc[ (X['average_rating'] > 3) & (X['average_rating'] <= 4), 'rating_between'] = "between 3 and 4"
X.loc[ (X['average_rating'] > 4) & (X['average_rating'] <= 5), 'rating_between'] = "between 4 and 5"

In [7]:
rating=pd.get_dummies(X['rating_between'])
lang=pd.get_dummies(X['language_code'])

In [8]:
features = pd.concat([rating,
                      lang,
                      X['average_rating'],
                      X['ratings_count']
    
], axis=1)

In [9]:
scaler=MinMaxScaler()

In [10]:
features=scaler.fit_transform(features)


In [11]:
from sklearn.cluster import KMeans

In [12]:
model=neighbors.NearestNeighbors()

In [13]:
model.fit(features)

NearestNeighbors()

In [14]:
dist, idlist = model.kneighbors(features)

In [15]:
def BookRecommender(book_name):
    book_list_name = []
    book_id = X[X['title'] == book_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(X.loc[newid].title)
    return book_list_name

In [16]:
new_books=BookRecommender('Seven Japanese Tales')

In [17]:
new_books

['Seven Japanese Tales',
 'A Frolic of His Own',
 'The Man Who Watched Trains Go By',
 "Foreign Correspondence: A Pen Pal's Journey from Down Under to All Over",
 'Love With the Proper Husband (Effingtons  #6)']

In [18]:
import joblib



In [19]:
pickle_model = joblib.dump(model,'book_recommender.pkl')